Importing some libraries

In [391]:
import pandas as pd

In [392]:
# URL of the data to be read from wiki
url_data = 'https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'

In [393]:
# Using pandas to read html
df_wiki = pd.read_html(url_data)

# Capture only the table data
df = df_wiki[0]


In [394]:
# Remove rows with "Not assigned"
df = df[df.Neighbourhood != 'Not assigned']


Editing the records to allow one or more neighborhood in one postal code area

In [395]:
# Unique postcode
postcodes = df.Postcode.unique()

In [396]:
new_df = []
for i in range(len(postcodes)):
    idx = df.loc[df['Postcode'] == postcodes[i]]
    str_found = pd.DataFrame(idx.Neighbourhood)
    [r, c] = str_found.shape
    neighborhood_name = []
    for j in range(r):
        neighborhood_name.append(str_found.iat[j,0])
    tmp = [postcodes[i], df.iloc[i]['Borough'], neighborhood_name]
    new_df.append(tmp)


In [397]:
new_df

[['M3A', 'North York', ['Parkwoods']],
 ['M4A', 'North York', ['Victoria Village']],
 ['M5A', 'Downtown Toronto', ['Harbourfront', 'Regent Park']],
 ['M6A', 'Downtown Toronto', ['Lawrence Heights', 'Lawrence Manor']],
 ['M9A', 'North York', ['Islington Avenue']],
 ['M1B', 'North York', ['Rouge', 'Malvern']],
 ['M3B', 'Etobicoke', ['Don Mills North']],
 ['M4B', 'Scarborough', ['Woodbine Gardens', 'Parkview Hill']],
 ['M5B', 'Scarborough', ['Ryerson', 'Garden District']],
 ['M6B', 'North York', ['Glencairn']],
 ['M9B',
  'East York',
  ['Cloverdale',
   'Islington',
   'Martin Grove',
   'Princess Gardens',
   'West Deane Park']],
 ['M1C', 'East York', ['Highland Creek', 'Rouge Hill', 'Port Union']],
 ['M3C', 'Downtown Toronto', ['Flemingdon Park', 'Don Mills South']],
 ['M4C', 'Downtown Toronto', ['Woodbine Heights']],
 ['M5C', 'North York', ['St. James Town']],
 ['M6C', 'Etobicoke', ['Humewood-Cedarvale']],
 ['M9C',
  'Etobicoke',
  ['Bloordale Gardens', 'Eringate', 'Markland Wood', 'O

In [398]:
new_df = pd.DataFrame(new_df, columns = ['Postcode','Borough' , 'Neighbourhood'])

In [400]:
new_df

,Postcode,Borough,Neighbourhood
0,M3A,North York,[Parkwoods]
1,M4A,North York,[Victoria Village]
2,M5A,Downtown Toronto,"[Harbourfront, Regent Park]"
3,M6A,Downtown Toronto,"[Lawrence Heights, Lawrence Manor]"
4,M9A,North York,[Islington Avenue]
5,M1B,North York,"[Rouge, Malvern]"
6,M3B,Etobicoke,[Don Mills North]
7,M4B,Scarborough,"[Woodbine Gardens, Parkview Hill]"
8,M5B,Scarborough,"[Ryerson, Garden District]"
9,M6B,North York,[Glencairn]


In [399]:
new_df.shape

(102, 3)